## This notebook is to do missing data imputation for trainset

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set()

pd.set_option('display.max_rows',2000)
pd.set_option('display.max_columns',50)
pd.set_option('display.width',1000)

from pprint import pprint
from feature_engine import missing_data_imputers as mdi

import warnings
warnings.filterwarnings("ignore")

### Load Data and Split

In [2]:
df = pd.read_csv('combined.csv')

In [3]:
df.head()

,county_code,state,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,rucc,urban_influence,economic_typology,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
0,8e686a7,fb8cab1,3876,408,24.583,NaN,18.380,0.945945,0.010761,0.026038,0.005685,0.005635,0.0,0.005935,0.000000,4.172,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Manufacturing-dependent,0.550,0.023,0.107,0.062,0.310,0.166,0.100,NaN,0.262,0.342,11.022908,NaN,NaN,217,NaN,NaN,0.471,0.218,0.190,0.083250,0.327984,0.389168,0.199599,10.008982,9.752342,577
1,d1b5fc5,842bd12,10224,1166,28.346,3.0,26.694,0.808959,0.002284,0.173426,0.001803,0.001493,0.0,0.012033,0.000000,7.714,"Nonmetro - Completely rural or less than 2,500...",Micropolitan not adjacent to a metro area,Recreation,0.545,0.040,0.324,0.201,0.253,0.102,0.066,0.070,0.200,0.148,9.904099,NaN,18.26,181,2009.0,1690.0,0.472,0.290,0.077,0.099099,0.202202,0.302302,0.396396,14.482893,3.013219,844
2,19a463b,2b7da97,27023,2927,21.641,9.0,31.028,0.956621,0.005141,0.013088,0.004728,0.004708,0.0,0.014809,0.000905,9.789,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore not adjacent to a metro/micro area and...,Recreation,0.508,0.063,0.129,0.032,0.296,0.215,0.087,0.070,0.187,0.258,11.011502,NaN,19.94,234,4650.0,1350.0,0.496,0.208,0.192,0.137725,0.387226,0.255489,0.219561,9.620346,10.873572,700
3,1711ab7,5029ed4,8735,1039,23.110,0.0,27.734,0.894835,0.000298,0.076576,0.000995,0.004297,0.0,0.022999,0.000000,13.735,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore not adjacent to a metro/micro area and...,Nonspecialized,0.545,0.038,0.207,0.084,0.255,0.182,0.104,0.081,0.169,0.274,12.845770,NaN,30.31,231,4420.0,1749.0,0.503,0.193,0.215,0.077154,0.308617,0.398798,0.215431,9.383225,10.870809,592
4,1eb4681,b795815,3681,365,21.985,2.0,19.673,0.923886,0.000496,0.042889,0.000000,0.013622,0.0,0.016905,0.002202,8.217,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Farm-dependent,0.474,0.034,0.177,0.075,0.324,0.118,0.113,NaN,0.178,0.340,11.565750,NaN,NaN,270,NaN,1249.0,0.516,0.222,0.230,0.092537,0.362189,0.346269,0.199005,9.543286,14.723926,444


In [4]:
df.tail()

,county_code,state,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,rucc,urban_influence,economic_typology,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
3133,dae1584,52acab4,44414,6275,35.026,254.0,30.569,0.726121,0.019125,0.054938,0.143854,0.005045,0.000301,0.050415,0.000201,13.606,"Nonmetro - Urban population of 2,500 to 19,999...",Micropolitan adjacent to a small metro area,Nonspecialized,0.401,0.048,0.300,0.131,0.333,0.259,0.117,0.08,0.091,0.297,10.200021,5.67,25.89,330,2779.0,2530.0,0.511,0.233,0.165,0.158683,0.332335,0.295409,0.213573,13.675599,11.802539,671
3134,b95e0ba,485e9af,16160,1871,31.922,NaN,33.899,0.663196,0.303469,0.015262,0.001817,0.001717,0.000000,0.004257,0.010282,20.052,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore adjacent to a small metro with town of...,Nonspecialized,0.450,0.059,0.254,0.062,0.355,0.195,0.111,0.10,NaN,0.333,10.897362,NaN,30.34,347,8620.0,8160.0,0.495,0.227,0.176,0.210159,0.419323,0.240040,0.130478,10.286142,13.278474,612
3135,f44348c,1dcfd4e,28688,4478,33.897,NaN,28.644,0.809853,0.011596,0.147624,0.001816,0.009389,0.003832,0.005427,0.010463,6.418,"Nonmetro - Urban population of 20,000 or more,...",Micropolitan not adjacent to a metro area,Recreation,0.696,0.034,0.231,0.132,0.144,0.096,0.039,0.11,0.331,0.117,12.726635,NaN,10.74,78,1229.0,959.0,0.451,0.170,0.094,0.051793,0.173307,0.288845,0.486056,9.229493,2.385150,1007
3136,c9450a6,52acab4,29234,4528,40.479,57.0,24.803,0.707838,0.030869,0.160475,0.025157,0.014104,0.000700,0.060857,0.000000,12.429,"Nonmetro - Urban population of 2,500 to 19,999...",Micropolitan not adjacent to a metro area,Mining-dependent,0.532,0.031,0.286,0.133,0.287,0.161,0.108,0.08,0.107,0.335,9.967003,NaN,18.86,349,2139.0,1849.0,0.501,0.237,0.131,0.137412,0.350050,0.243731,0.268806,16.695002,9.758586,713
3137,c229ad2,78e8330,4036,327,18.052,NaN,22.122,0.967377,0.001192,0.012517,0.013282,0.003944,0.000000,0.001689,0.000000,6.621,"Nonmetro - Completely rural or less than 2,500...",Micropolitan not adjacent to a metro area,Farm-dependent,0.562,0.025,0.158,0.074,0.358,0.130,0.089,NaN,0.202,0.295,9.667330,NaN,NaN,260,NaN,NaN,0.489,0.236,0.217,0.110000,0.338000,0.311000,0.241000,10.982154,7.487832,593


In [5]:
df.dtypes

county_code                                    object
state                                          object
population                                      int64
renter_occupied_households                      int64
pct_renter_occupied                           float64
evictions                                     float64
rent_burden                                   float64
pct_white                                     float64
pct_af_am                                     float64
pct_hispanic                                  float64
pct_am_ind                                    float64
pct_asian                                     float64
pct_nh_pi                                     float64
pct_multiple                                  float64
pct_other                                     float64
poverty_rate                                  float64
rucc                                           object
urban_influence                                object
economic_typology           

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3138 entries, 0 to 3137
Data columns (total 45 columns):
county_code                                   3138 non-null object
state                                         3138 non-null object
population                                    3138 non-null int64
renter_occupied_households                    3138 non-null int64
pct_renter_occupied                           3138 non-null float64
evictions                                     2498 non-null float64
rent_burden                                   3138 non-null float64
pct_white                                     3138 non-null float64
pct_af_am                                     3138 non-null float64
pct_hispanic                                  3138 non-null float64
pct_am_ind                                    3138 non-null float64
pct_asian                                     3138 non-null float64
pct_nh_pi                                     3138 non-null float64
pct_multiple   

In [7]:
df.describe(include='all')

,county_code,state,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,rucc,urban_influence,economic_typology,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
count,3138,3138,3.138000e+03,3.138000e+03,3138.000000,2498.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138,3138,3138,3138.000000,3138.000000,3133.000000,3133.000000,3133.000000,2705.000000,3133.000000,2912.000000,2219.000000,3133.000000,3105.000000,1250.000000,2771.000000,3138.000000,2917.000000,2933.000000,3133.000000,3133.000000,3133.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000
unique,3138,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,12,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,f564699,1b0d913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nonmetro - Urban population of 2,500 to 19,999...",Small-in a metro area with fewer than 1 millio...,Nonspecialized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,592,732,1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.008165e+05,1.396144e+04,28.565132,363.391914,28.514306,0.773394,0.088899,0.088337,0.017103,0.012534,0.000773,0.018076,0.000884,12.203356,NaN,NaN,NaN,0.471619,0.062445,0.217518,0.087144,0.305599,0.212639,0.107099,0.083530,0.164986,0.278452,11.626925,5.773744,21.546496,276.852135,3469.825848,2564.363450,0.499965,0.228692,0.167473,0.145687,0.347500,0.302631,0.204183,11.686458,10.402913,700.496176
std,NaN,NaN,3.229873e+05,5.403000e+04,8.065512,1356.683376,4.567738,0.198322,0.143953,0.135010,0.069948,0.026358,0.003976,0.017013,0.001628,5.732980,NaN,NaN,NaN,0.072493,0.022685,0.066169,0.040142,0.042400,0.063025,0.022506,0.021079,0.051633,0.052944,1.530516,4.682077,10.413872,58.834698,2666.067220,2146.460781,0.022426,0.033283,0.043080,0.066404,0.070716,0.051685,0.090151,2.660699,2.781861,175.806214
min,NaN,NaN,1.170000e+02,1.500000e+01,7.279000,-1.000000,9.909000,0.009547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.186000,0.012000,0.039000,0.013000,0.131000,0.031000,0.033000,0.030000,0.032000,0.103000,6.965724,-0.340000,3.140000,76.000000,340.000000,190.000000,0.286000,0.082000,0.036000,0.015984,0.074297,0.114458,0.018924,3.654080,0.961076,351.000000
25%,NaN,NaN,1.106575e+04,1.142750e+03,23.107000,4.000000,25.955250,0.657212,0.005606,0.018718,0.001189,0.002402,0.000000,0.009741,0.000000,8.063250,NaN,NaN,NaN,0.425000,0.046000,0.169000,0.058000,0.284000,0.170000,0.092000,0.070000,0.129000,0.245000,10.440005,2.682500,14.050000,234.000000,1870.000000,1379.000000,0.496000,0.209000,0.140000,0.095522,0.303326,0.266733,0.141752,10.063759,8.561913,586.000000
50%,NaN,NaN,2.587950e+04,2.812000e+03,27.289500,30.000000,28.745000,0.846237,0.020915,0.037449,0.002596,0.005386,0.000000,0.014736,0.000301,11.335000,NaN,NaN,NaN,0.472000,0.060000,0.215000,0.077000,0.307000,0.208000,0.106000,0.080000,0.163000,0.281000,11.889621,4.575000,19.910000,271.000000,2719.000000,1960.000000,0.504000,0.228000,0.1640

In [8]:
df.shape

(3138, 45)

### Treat Missing Values

In [9]:
df.isnull().sum()

county_code                                      0
state                                            0
population                                       0
renter_occupied_households                       0
pct_renter_occupied                              0
evictions                                      640
rent_burden                                      0
pct_white                                        0
pct_af_am                                        0
pct_hispanic                                     0
pct_am_ind                                       0
pct_asian                                        0
pct_nh_pi                                        0
pct_multiple                                     0
pct_other                                        0
poverty_rate                                     0
rucc                                             0
urban_influence                                  0
economic_typology                                0
pct_civilian_labor             

In [10]:
imputer = mdi.MeanMedianImputer(imputation_method='mean',variables=None)

In [11]:
imputer.fit(df)

MeanMedianImputer(imputation_method='mean',
                  variables=['population', 'renter_occupied_households',
                             'pct_renter_occupied', 'evictions', 'rent_burden',
                             'pct_white', 'pct_af_am', 'pct_hispanic',
                             'pct_am_ind', 'pct_asian', 'pct_nh_pi',
                             'pct_multiple', 'pct_other', 'poverty_rate',
                             'pct_civilian_labor', 'pct_unemployment',
                             'pct_uninsured_adults', 'pct_uninsured_children',
                             'pct_adult_obesity', 'pct_adult_smoking',
                             'pct_diabetes', 'pct_low_birthweight',
                             'pct_excessive_drinking',
                             'pct_physical_inactivity',
                             'air_pollution_particulate_matter_value',
                             'homicides_per_100k',
                             'motor_vehicle_crash_deaths_per_100k',

In [12]:
df1 = imputer.transform(df)

In [13]:
df1.isnull().sum()

county_code                                   0
state                                         0
population                                    0
renter_occupied_households                    0
pct_renter_occupied                           0
evictions                                     0
rent_burden                                   0
pct_white                                     0
pct_af_am                                     0
pct_hispanic                                  0
pct_am_ind                                    0
pct_asian                                     0
pct_nh_pi                                     0
pct_multiple                                  0
pct_other                                     0
poverty_rate                                  0
rucc                                          0
urban_influence                               0
economic_typology                             0
pct_civilian_labor                            0
pct_unemployment                        

In [14]:
df1.shape

(3138, 45)

### Save a copy

In [15]:
#df1.to_csv('combinedfilled.csv',index=False)

In [16]:
df1.head()

,county_code,state,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,rucc,urban_influence,economic_typology,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
0,8e686a7,fb8cab1,3876,408,24.583,363.391914,18.380,0.945945,0.010761,0.026038,0.005685,0.005635,0.0,0.005935,0.000000,4.172,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Manufacturing-dependent,0.550,0.023,0.107,0.062,0.310,0.166,0.100,0.08353,0.262,0.342,11.022908,5.773744,21.546496,217,3469.825848,2564.36345,0.471,0.218,0.190,0.083250,0.327984,0.389168,0.199599,10.008982,9.752342,577
1,d1b5fc5,842bd12,10224,1166,28.346,3.000000,26.694,0.808959,0.002284,0.173426,0.001803,0.001493,0.0,0.012033,0.000000,7.714,"Nonmetro - Completely rural or less than 2,500...",Micropolitan not adjacent to a metro area,Recreation,0.545,0.040,0.324,0.201,0.253,0.102,0.066,0.07000,0.200,0.148,9.904099,5.773744,18.260000,181,2009.000000,1690.00000,0.472,0.290,0.077,0.099099,0.202202,0.302302,0.396396,14.482893,3.013219,844
2,19a463b,2b7da97,27023,2927,21.641,9.000000,31.028,0.956621,0.005141,0.013088,0.004728,0.004708,0.0,0.014809,0.000905,9.789,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore not adjacent to a metro/micro area and...,Recreation,0.508,0.063,0.129,0.032,0.296,0.215,0.087,0.07000,0.187,0.258,11.011502,5.773744,19.940000,234,4650.000000,1350.00000,0.496,0.208,0.192,0.137725,0.387226,0.255489,0.219561,9.620346,10.873572,700
3,1711ab7,5029ed4,8735,1039,23.110,0.000000,27.734,0.894835,0.000298,0.076576,0.000995,0.004297,0.0,0.022999,0.000000,13.735,"Nonmetro - Urban population of 2,500 to 19,999...",Noncore not adjacent to a metro/micro area and...,Nonspecialized,0.545,0.038,0.207,0.084,0.255,0.182,0.104,0.08100,0.169,0.274,12.845770,5.773744,30.310000,231,4420.000000,1749.00000,0.503,0.193,0.215,0.077154,0.308617,0.398798,0.215431,9.383225,10.870809,592
4,1eb4681,b795815,3681,365,21.985,2.000000,19.673,0.923886,0.000496,0.042889,0.000000,0.013622,0.0,0.016905,0.002202,8.217,"Nonmetro - Completely rural or less than 2,500...",Noncore adjacent to micro area and does not co...,Farm-dependent,0.474,0.034,0.177,0.075,0.324,0.118,0.113,0.08353,0.178,0.340,11.565750,5.773744,21.546496,270,3469.825848,1249.00000,0.516,0.222,0.230,0.092537,0.362189,0.346269,0.199005,9.543286,14.723926,444


In [17]:
df1.describe(include='all')

,county_code,state,population,renter_occupied_households,pct_renter_occupied,evictions,rent_burden,pct_white,pct_af_am,pct_hispanic,pct_am_ind,pct_asian,pct_nh_pi,pct_multiple,pct_other,poverty_rate,rucc,urban_influence,economic_typology,pct_civilian_labor,pct_unemployment,pct_uninsured_adults,pct_uninsured_children,pct_adult_obesity,pct_adult_smoking,pct_diabetes,pct_low_birthweight,pct_excessive_drinking,pct_physical_inactivity,air_pollution_particulate_matter_value,homicides_per_100k,motor_vehicle_crash_deaths_per_100k,heart_disease_mortality_per_100k,pop_per_dentist,pop_per_primary_care_physician,pct_female,pct_below_18_years_of_age,pct_aged_65_years_and_older,pct_adults_less_than_a_high_school_diploma,pct_adults_with_high_school_diploma,pct_adults_with_some_college,pct_adults_bachelors_or_higher,birth_rate_per_1k,death_rate_per_1k,gross_rent
count,3138,3138,3.138000e+03,3.138000e+03,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138,3138,3138,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000,3138.000000
unique,3138,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,12,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,f564699,1b0d913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Nonmetro - Urban population of 2,500 to 19,999...",Small-in a metro area with fewer than 1 millio...,Nonspecialized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,592,732,1234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.008165e+05,1.396144e+04,28.565132,363.391914,28.514306,0.773394,0.088899,0.088337,0.017103,0.012534,0.000773,0.018076,0.000884,12.203356,NaN,NaN,NaN,0.471619,0.062445,0.217518,0.087144,0.305599,0.212639,0.107099,0.083530,0.164986,0.278452,11.626925,5.773744,21.546496,276.852135,3469.825848,2564.363450,0.499965,0.228692,0.167473,0.145687,0.347500,0.302631,0.204183,11.686458,10.402913,700.496176
std,NaN,NaN,3.229873e+05,5.403000e+04,8.065512,1210.404460,4.567738,0.198322,0.143953,0.135010,0.069948,0.026358,0.003976,0.017013,0.001628,5.732980,NaN,NaN,NaN,0.072493,0.022685,0.066116,0.040110,0.042367,0.058514,0.022488,0.020306,0.043416,0.052902,1.522444,2.954354,9.785767,58.834698,2570.440739,2075.141335,0.022408,0.033257,0.043046,0.066404,0.070716,0.051685,0.090151,2.660699,2.781861,175.806214
min,NaN,NaN,1.170000e+02,1.500000e+01,7.279000,-1.000000,9.909000,0.009547,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.186000,0.012000,0.039000,0.013000,0.131000,0.031000,0.033000,0.030000,0.032000,0.103000,6.965724,-0.340000,3.140000,76.000000,340.000000,190.000000,0.286000,0.082000,0.036000,0.015984,0.074297,0.114458,0.018924,3.654080,0.961076,351.000000
25%,NaN,NaN,1.106575e+04,1.142750e+03,23.107000,8.000000,25.955250,0.657212,0.005606,0.018718,0.001189,0.002402,0.000000,0.009741,0.000000,8.063250,NaN,NaN,NaN,0.425000,0.046000,0.169000,0.058250,0.284000,0.176000,0.092000,0.070000,0.144000,0.245000,10.449944,5.773744,14.885000,234.000000,1930.000000,1429.000000,0.496000,0.209000,0.140000,0.095522,0.303326,0.266733,0.141752,10.063759,8.561913,586.000000
50%,NaN,NaN,2.587950e+04,2.812000e+03,27.289500,73.000000,28.745000,0.846237,0.020915,0.037449,0.002596,0.005386,0.000000,0.014736,0.000301,11.335000,NaN,NaN,NaN,0.472000,0.060000,0.215000,0.078000,0.307000,0.212639,0.106000,0.080000,0.164986,0.281000,11.858134,5.773744,21.546496,271.000000,2879.000000,2039.000000,0.504000,0.228000,0.16400